In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Installation and maintenance cost (IM6, CB8, OS3, IM4, IM5)

In [2]:
x_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_train_scale.csv', 
                    delimiter = ',', usecols = (10,11,12,13,14), skip_header=1)
y_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_train_scale.csv', 
                   delimiter =',', usecols = (23,24,25,26,27,28,29), skip_header=1)
x_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_cv_scale.csv', 
                    delimiter = ',', usecols = (10,11,12,13,14), skip_header=1)
y_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_cv_scale.csv',
                      delimiter =',', usecols = (23,24,25,26,27,28,29),skip_header=1)

In [3]:
y_whole = np.vstack ((y_1, y_2)).astype(np.float32)
x_whole = np.vstack ((x_1, x_2)).astype(np.float32)
x_traind = x_1
x_test = x_2
y_traind = y_1
y_test = y_2
x_train, x_cv, y_train, y_cv = train_test_split (x_1,y_1,test_size=0.13)

In [7]:
batch_size = 70
num_classes = 7

model = Sequential ()
model.add (Dense (units = 19, input_dim = 5))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 19))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 19))
#model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 16))
model.add (Dense(num_classes, activation = 'tanh'))

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])

History = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=60,
                    verbose=1, validation_data=(x_cv, y_cv))

print(model.evaluate (x_test, y_test, batch_size = 70, verbose =1))

#help (model.evaluate)
#this is the output units

y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv, y_pred_test)).astype(np.float32)

Train on 67 samples, validate on 11 samples
Epoch 1/60
67/67 [==============================] - ETA: 0s - loss: 2.4495 - binary_accuracy: 0.5714 - val_loss: 3.1411 - val_binary_accuracy: 0.5584
Epoch 2/60
67/67 [==============================] - ETA: 0s - loss: 2.3274 - binary_accuracy: 0.5800 - val_loss: 2.9718 - val_binary_accuracy: 0.5584
Epoch 3/60
67/67 [==============================] - ETA: 0s - loss: 2.2947 - binary_accuracy: 0.5821 - val_loss: 2.9478 - val_binary_accuracy: 0.5584
Epoch 4/60
67/67 [==============================] - ETA: 0s - loss: 2.2655 - binary_accuracy: 0.5821 - val_loss: 3.1266 - val_binary_accuracy: 0.5455
Epoch 5/60
67/67 [==============================] - ETA: 0s - loss: 2.1292 - binary_accuracy: 0.5864 - val_loss: 2.9653 - val_binary_accuracy: 0.5455
Epoch 6/60
67/67 [==============================] - ETA: 0s - loss: 2.0441 - binary_accuracy: 0.5864 - val_loss: 2.7783 - val_binary_accuracy: 0.5325
Epoch 7/60
67/67 [==============================] - ETA:

Epoch 56/60
67/67 [==============================] - ETA: 0s - loss: 1.2432 - binary_accuracy: 0.6418 - val_loss: 2.0974 - val_binary_accuracy: 0.6104
Epoch 57/60
67/67 [==============================] - ETA: 0s - loss: 1.1759 - binary_accuracy: 0.6439 - val_loss: 2.0976 - val_binary_accuracy: 0.6104
Epoch 58/60
67/67 [==============================] - ETA: 0s - loss: 1.1724 - binary_accuracy: 0.6482 - val_loss: 2.0977 - val_binary_accuracy: 0.6234
Epoch 59/60
67/67 [==============================] - ETA: 0s - loss: 1.1700 - binary_accuracy: 0.6567 - val_loss: 1.9532 - val_binary_accuracy: 0.6234
Epoch 60/60
20/20 [==============================] - ETA: 0s
[0.85079562664031982, 0.65714287757873535]


### t(x) for labelling

In [8]:
#creating 100 random numbers
y_pred = y_pred_whole

f = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Keras/f_random.csv', 
                    delimiter = ',').reshape(100,1)
def t(x):
    return f.item (x)

def k (x):
    k = np.zeros ((98,7));
    i = 0;
    j = 0;
    for i in range (98):
        for j in range (7):
            if y_pred.item((i,j))<=t(x):
                np.put(k,[7*i+j],[1]);
    return k

g = np.zeros((98,1))
for x in range (100):
    m = np.sum(k(x), axis = 1).reshape(98,1);
    #print (m.shape)
    g = np.hstack((g,m));
g = np.delete(g,0,1)


i = 0
j = 0
m = np.zeros((98,100))
for i in range (98):
    min =100
    for j in range (100):
        if g.item ((i,j))<=min:
            min = g.item((i,j))
    for j in range (100):
        if g.item ((i,j))==min:
            np.put(m,[100*i+j],[1])
            
i = 0
j = 0
w = np.zeros((98,100))
for i in range (0,98):
    for j in range (0, 100):
        w_ = m.item((i,j))*t(j)
        np.put(w,[100*i+j],[w_])

        
cnt = np.count_nonzero(w, axis=1).reshape(98,1)
re_cnt = cnt[::-1]
w_total = np.sum(w, axis = 1).reshape(98,1)
w_avg = w_total/re_cnt
A = np.hstack([y_pred, np.ones((98,1))])
w1,w2,w3,w4,w5,w6,w7,b = np.linalg.lstsq(A,w_avg)[0]
print (w1,w2,w3,w4,w5,w6,w7,b)


def y_label (y_pred):
    
    I = y_pred.shape[0];
    J = y_pred.shape[1];
    
    x1_Y = y_pred [:,0].reshape(I,1);
    x2_Y = y_pred [:,1].reshape(I,1);
    x3_Y = y_pred [:,2].reshape(I,1);
    x4_Y = y_pred [:,3].reshape(I,1);
    x5_Y = y_pred [:,4].reshape(I,1);
    x6_Y = y_pred [:,5].reshape(I,1);
    x7_Y = y_pred [:,6].reshape(I,1);
    
    c_x = y_pred.reshape(I,J);
    y_thre = np.zeros((I,1));
    y_label = np.zeros((I,J));
    
    i = 0;
    j = 0;
    
    for i in range (I):
        x1 = x1_Y.item(i,0);
        x2 = x2_Y.item(i,0);
        x3 = x3_Y.item(i,0);
        x4 = x4_Y.item(i,0);
        x5 = x5_Y.item(i,0);
        x6 = x6_Y.item(i,0);
        x7 = x7_Y.item(i,0);
        y_tx = w1*x1+w2*x2+w3*x3+w4*x4+w5*x5+w6*x6+w7*x7+b;
        np.put (y_thre, [i], [y_tx]);
        
    for i in range (I):
        for j in range (J):
            threshold = y_thre.item(i,0);
            if c_x.item(i,j) >= threshold:
                np.put(y_label, [J*i+j], [1])
    
    return (y_label)


y_train_label = y_label(y_pred_train)
y_cv_label = y_label(y_pred_cv)
y_test_label = y_label(y_pred_test)
y_t_label = np.vstack((y_train_label,y_cv_label))
y_t = np.vstack((y_train,y_cv))

from sklearn.metrics import hamming_loss
from sklearn.metrics import coverage_error
print('hamming_loss_train',hamming_loss(y_train, y_train_label))
print('hamming_loss_cv',hamming_loss(y_cv, y_cv_label))
print('hamming_loss_test',hamming_loss(y_test, y_test_label))
print('hamming_loss_traindataset',hamming_loss(y_t, y_t_label))

[ 0.63994964] [-1.39928835] [ 0.34511126] [-0.26644971] [ 1.34175148] [ 0.65957136] [-0.40888216] [-0.31021982]
hamming_loss_train 0.3795309168443497
hamming_loss_cv 0.4935064935064935
hamming_loss_test 0.36428571428571427
hamming_loss_traindataset 0.3956043956043956


In [9]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('model_app_install_cost_a.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_app_install_cost_a.h5')
print ('Saved model to disk')

Saved model to disk


In [3]:
#load json and create model
json_file = open('model_app_install_cost_a.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model_app_install_cost_a.h5')

In [4]:
x_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv', 
                    delimiter = ',', usecols = (10,11,12,13,14), skip_header=1)
y_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv',
                      delimiter =',', usecols = (23,24,25,26,27,28,29),skip_header=1)



loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_test, y_test, batch_size = 50))
y_pred_test = loaded_model.predict (x_test)

x1 = y_pred_test [:,0].reshape(14,1);
x2 = y_pred_test [:,1].reshape(14,1);
x3 = y_pred_test [:,2].reshape(14,1);
x4 = y_pred_test [:,3].reshape(14,1);
x5 = y_pred_test [:,4].reshape(14,1);
x6 = y_pred_test [:,5].reshape(14,1);
x7 = y_pred_test [:,6].reshape(14,1);

i = 0;
label_cf = np.zeros((14,1));
for i in range (14):
    y_tx = -1.12*x1+0.64*x2-0.13*x3+2*x4+0.31*x5+0.11*x6+0.33*x7+0.1;
    np.put(label_cf, [i], [y_tx]);
    label = np.hstack((y_pred_test, label_cf)).reshape((14,8))
    
    
i = 0;
j = 0;
cf = np.zeros((14,7))
for i in range (14):
    for j in range (7):
        cf_1 = y_pred_test.item((i,j))-label_cf.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [7*i+j], [cf_2]);
        
np.savetxt('y_cost_APPinstall_cost.csv', cf, delimiter=',') 

14/14 [==============================] - ETA: 0s
[278.82583618164063, 0.6428571343421936]


In [5]:
x_whole = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_whole_scale.csv', 
                    delimiter = ',', usecols = (10,11,12,13,14), skip_header=1)
y_whole = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a_whole_scale.csv',
                      delimiter =',', usecols = (23,24,25,26,27,28,29),skip_header=1)


loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_whole, y_whole, batch_size = 50))
y_pred_test = loaded_model.predict (x_whole)

x1 = y_pred_test [:,0].reshape(98,1);
x2 = y_pred_test [:,1].reshape(98,1);
x3 = y_pred_test [:,2].reshape(98,1);
x4 = y_pred_test [:,3].reshape(98,1);
x5 = y_pred_test [:,4].reshape(98,1);
x6 = y_pred_test [:,5].reshape(98,1);
x7 = y_pred_test [:,6].reshape(98,1);

i = 0;
label_cf = np.zeros((98,1));
for i in range (98):
    y_tx = -1.12*x1+0.64*x2-0.13*x3+2*x4+0.31*x5+0.11*x6+0.33*x7+0.1;
    np.put(label_cf, [i], [y_tx]);
    label = np.hstack((y_pred_test, label_cf)).reshape((98,8))
    
    
i = 0;
j = 0;
cf = np.zeros((98,7))
for i in range (98):
    for j in range (7):
        cf_1 = y_pred_test.item((i,j))-label_cf.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [7*i+j], [cf_2]);
        
np.savetxt('APPinstall_cost_total.csv', cf, delimiter=',') 

98/98 [==============================] - ETA: 0s
[1.1888981133091205, 0.654518972854225]
